# 数据预处理过程
## 1.读取数据

从文件 `'附件2.xlsx'` 中读取两个 Excel 工作表（`sheet1` 和 `sheet2`），分别存储在 `df_crops` 和 `df_land` 数据框中。  
`df_crops` 表包含作物的种植信息，`df_land` 表包含作物的详细销售和种植条件信息。

## 2.填充缺失值

对 `df_crops` 数据框进行缺失值填充操作，使用前向填充法 (`ffill`)，以确保数据完整性。  
这是因为一些单元格可能包含缺失值，需要使用前一个有效值进行填充。

## 3.合并两个表格

通过作物编号（`作物编号`）将 `df_crops` 和 `df_land` 进行合并操作。  
使用 `pd.merge()` 函数，采用 `left join`，确保在 `df_crops` 中存在的所有作物编号都能在合并后的数据框中保留，而来自 `df_land` 的附加信息被添加到相应的行中。

## 4.计算销售量

新增一列 `'销售量/斤'`，表示每种作物的总销售量（单位：斤）。  
销售量计算公式为：`销售量/斤 = 种植面积/亩 * 亩产量/斤`，即每亩地的产量乘以种植的面积。  
这是一个直接计算的过程，假设所有产量均能以全额销售出去。

## 5.筛选出关心的列

从合并后的数据框中选择与分析相关的列，如`种植地块`、`作物编号`、`作物名称`、`种植面积`、`种植季次`、`亩产量`等。  
使用一个 `selected_columns` 列表来明确我们需要的列。

## 6.保存整理后的数据

将整理后的数据（即 `sales_volume_df` 数据框）保存到一个新的 Excel 文件 `整理后附件2数据.xlsx` 中。  

## 6.按作物名称汇总销售量

使用 `groupby()` 函数按 `作物名称` 对数据进行分组，并使用 `agg()` 函数汇总每种作物的总销售量（`销售量/斤`）。  
汇总后的数据框（`total_sales_volume_df`）保存到新的 Excel 文件 `各作物销售量.xlsx` 中。

## 8.计算销售价格

创建一个 `calculate_sales_price` 函数，用于基于每种作物的销售单价区间（例如“10-15”元/斤）随机生成一个销售价格。  
使用 `apply()` 函数将该计算函数应用于数据框中的每一行。

## 9.按作物名称计算平均销售价格和总销售量

使用 `groupby()` 和 `agg()` 函数，按 `作物名称` 汇总计算每种作物的平均销售单价（`销售单价/(元/斤)`）和总销售量（`销售量/斤`）。  
汇总后的数据框（`sales_prices_df`）进一步清洗数据并命名列，以便于输出和分析。

## 10.保存最终结果

将汇总后的作物平均销售价格和总销售量数据保存到一个新 Excel 文件 `各作物聚合后销售量与价格.xlsx` 中。  
这是最终的处理结果文件，提供了各作物的销售单价和销售量汇总信息。


In [1]:
import pandas as pd
import numpy as np

file_path = './附件2.xlsx'

# 读取 Excel 表格中的两个 sheet
df_crops = pd.read_excel(file_path, sheet_name=0)
df_land = pd.read_excel(file_path, sheet_name=1)

# 填充缺失值
df_crops.fillna(method='ffill', inplace=True)

# 合并两个表格的数据，依据作物编号
merged_df = pd.merge(df_crops, df_land, on="作物编号", how='left')

# 计算销售量（生产量就是销售量），销售量 = 种植面积/亩 * 亩产量/斤
merged_df['销售量/斤'] = merged_df['种植面积/亩'] * merged_df['亩产量/斤']

# 筛选出关心的列
selected_columns = ['种植地块', '作物编号', '作物名称_x', '作物类型', '种植面积/亩', 
                    '种植季次_x', '地块类型', '亩产量/斤', '种植成本/(元/亩)', '销售单价/(元/斤)', '销售量/斤']
sales_volume_df = merged_df[selected_columns]

# 保存整理后的数据
sales_volume_df.to_excel('./整理后附件2数据.xlsx', index=False)

# 按作物名称汇总销售量
total_sales_volume_df = sales_volume_df.groupby('作物名称_x').agg({'销售量/斤': 'sum'}).reset_index()
total_sales_volume_df.to_excel('./各作物销售量.xlsx', index=False)

# 计算销售价格
def calculate_sales_price(price_range):
    min_price, max_price = map(float, price_range.split('-'))
    return min_price + np.random.rand() * (max_price - min_price)

# 应用函数计算销售价格
sales_volume_df['销售单价/(元/斤)'] = sales_volume_df['销售单价/(元/斤)'].apply(calculate_sales_price)

# 按作物名称计算平均销售价格和总销售量
sales_prices_df = sales_volume_df.groupby('作物名称_x').agg({'销售单价/(元/斤)': 'mean', '销售量/斤': 'sum'}).reset_index()

# 重命名列名
sales_prices_df.columns = ['作物名称', '销售单价/(元/斤)', '销售量/斤']

# 保存聚合后的数据
sales_prices_df.to_excel('./各作物聚合后销售量与价格.xlsx', index=False)


/var/folders/ng/h1_4zcsj2qq895j6xpxvjyqh0000gn/T/ipykernel_22537/2238197556.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sales_volume_df['销售单价/(元/斤)'] = sales_volume_df['销售单价/(元/斤)'].apply(calculate_sales_price)


## 优化问题的步骤和数学公式

### 目标

优化目标是**最大化净收入**并**最小化因超产导致的滞销损失**。具体来说，我们需要在给定的地块上选择合适的作物种植方案，以满足各种约束条件，同时实现收益最大化。

### 决策变量

定义决策变量 $x_{i,j,y}$，表示在地块 $i$ 上种植作物 $j$ 在年份 $y$ 的种植面积（单位：亩）。

### 目标函数

优化问题的目标函数可以表示为：

$$
\text{Maximize:} \quad Z = \sum_{i} \sum_{j} \sum_{y} \left( P_j \cdot Y_j \cdot x_{i,j,y} - C_j \cdot x_{i,j,y} - W_j \cdot E_j \right)
$$

其中：
- $Z$ 表示总净收益，即我们希望最大化的目标值。
- $P_j$ 表示作物 $j$ 的销售单价（单位：元/斤），代表作物 $j$ 的市场售价。
- $Y_j$ 表示作物 $j$ 的亩产量（单位：斤/亩），即每亩地可以收获的作物产量。
- $x_{i,j,y}$ 表示在地块 $i$ 上种植作物 $j$ 在年份 $y$ 的种植面积（单位：亩）。
- $C_j$ 表示作物 $j$ 的种植成本（单位：元/亩），包括种子、肥料、劳动力等相关成本。
- $W_j$ 表示作物 $j$ 的滞销单价（单位：元/斤），即未能出售的作物可能产生的损失。
- $E_j$ 表示作物 $j$ 的滞销量（单位：斤），即超过预期销售量的产量。

目标函数的意义是：**最大化每个地块在每年种植不同作物所获得的净收入，并考虑滞销造成的潜在损失。**

### 约束条件

1. **地块面积约束**：每个地块的种植面积不能超过其总面积。

$$
\sum_{j} x_{i,j,y} \leq A_i, \quad \forall i, y
$$

其中：
- $A_i$ 表示地块 $i$ 的总面积（单位：亩），即该地块可用于种植的最大面积。

该约束确保每块地的总种植面积不能超过其可用面积。

2. **最小种植面积约束**：非零种植面积不能小于某个最小面积。

$$
x_{i,j,y} \geq M \cdot \text{I}(x_{i,j,y} > 0), \quad \forall i, j, y
$$

其中：
- $M$ 表示最小种植面积（单位：亩），如 5 亩。
- $\text{I}(x_{i,j,y} > 0)$ 是指示函数，当 $x_{i,j,y} > 0$ 时，值为 1，否则为 0。

该约束确保每个种植区域的面积达到一个合理的规模。

3. **三年内至少种植一次豆类作物**：在每个地块上，三年内至少种植一次豆类作物。

$$
\sum_{y=y_0}^{y_0+2} \sum_{j \in \text{Beans}} x_{i,j,y} \geq 1, \quad \forall i
$$

其中：
- $\text{Beans}$ 表示豆类作物的集合。

该约束确保作物轮作制度得以实现，避免土壤贫瘠化。

4. **销售量约束**（可选）：若作物 $j$ 的产量超过预期销售量，则超过部分无法销售。

$$
Y_j \cdot x_{i,j,y} \leq S_j, \quad \forall i, j, y
$$

其中：
- $S_j$ 表示作物 $j$ 的预期销售量（单位：斤）。

该约束确保产量不会超出市场需求，避免不必要的浪费。

### 优化方法

采用线性规划方法，通过 `scipy.optimize.linprog` 函数求解问题。目标函数的系数、约束矩阵和约束条件都将被转化为优化模型中的输入参数。

### 求解流程

1. **构建目标函数**：
   - 计算每种作物在每个地块和每年种植的净收入和滞销损失，构成目标函数的系数。

2. **建立约束条件**：
   - 构建地块面积约束、最小种植面积约束、三年内至少种植一次豆类作物的约束，以及可选的销售量约束。

3. **求解优化问题**：
   - 使用 `linprog` 函数，输入目标函数和约束条件，求解最优种植方案。

4. **输出结果**：
   - 根据求解结果，输出每个地块在每年、每季的作物种植面积，以及相应的收益分析。

### 结果

优化结果将提供一个合理的作物种植方案，最大化收益并满足所有约束条件，便于农业生产决策的制定和优化。


In [2]:
import pandas as pd
import numpy as np
from scipy.optimize import linprog
import random

# 读取四个表格的数据
planting_data_file = './2023年的种植数据与销售量.xlsx'
crop_sales_file = './各作物聚合后销售量与价格.xlsx'
crop_land_file = './各作物适合种植的地块类型与季别.xlsx'
land_info_file = './乡村的现有耕地.xlsx'



In [3]:
# 加载表格
planting_data = pd.read_excel(planting_data_file)
crop_sales_data = pd.read_excel(crop_sales_file)
crop_land_data = pd.read_excel(crop_land_file)
land_info_data = pd.read_excel(land_info_file)

# 提取2023年各作物的销售量
crop_sales_2023 = crop_sales_data.set_index('作物名称')['销售量/斤'].to_dict()

# 提取地块信息
land_names = land_info_data['地块名称'].unique().tolist()

# 根据作物适宜种植的季节筛选适合第一季和第二季的作物
def get_suitable_crops_for_season(season):
    if season == '第一季':
        suitable_crops = crop_land_data[crop_land_data[f'水浇地{season}'] == 1]['作物名称'].tolist() + \
                         crop_land_data[crop_land_data[f'普通大棚{season}'] == 1]['作物名称'].tolist() + \
                         crop_land_data[crop_land_data[f'智慧大棚{season}'] == 1]['作物名称'].tolist() + \
                         crop_land_data[crop_land_data[f'平旱地'] == 1]['作物名称'].tolist() + \
                         crop_land_data[crop_land_data[f'梯田'] == 1]['作物名称'].tolist() + \
                         crop_land_data[crop_land_data[f'山坡地'] == 1]['作物名称'].tolist() + \
                         crop_land_data[crop_land_data[f'水浇地'] == 1]['作物名称'].tolist()
    else:  # 第二季
        suitable_crops = crop_land_data[crop_land_data[f'水浇地{season}'] == 1]['作物名称'].tolist() + \
                         crop_land_data[crop_land_data[f'普通大棚{season}'] == 1]['作物名称'].tolist() + \
                         crop_land_data[crop_land_data[f'智慧大棚{season}'] == 1]['作物名称'].tolist()
    
    return suitable_crops

suitable_crops_first_season = get_suitable_crops_for_season('第一季')
suitable_crops_second_season = get_suitable_crops_for_season('第二季')

# 过滤不适合的作物，只保留适合种植的作物
crops_first_season = [crop for crop in planting_data['作物名称_x'].unique() if crop in suitable_crops_first_season]
crops_second_season = [crop for crop in planting_data['作物名称_x'].unique() if crop in suitable_crops_second_season]

# 第二季可用的地块
land_names_second_season = ['D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 
                            'E1', 'E2', 'E3', 'E4', 'E5', 'E6', 'E7', 'E8', 'E9', 'E10', 'E11', 'E12', 'E13', 'E14', 'E15', 'E16', 
                            'F1', 'F2', 'F3', 'F4']

# 随机选择作物的数量，减少计算量
max_crops_per_land = 9


In [4]:
# 为每个地块创建作物映射
def create_land_crop_mapping(land_names, suitable_crops):
    land_crop_mapping = {}
    for land in land_names:
        if len(suitable_crops) > max_crops_per_land:
            selected_crops = random.sample(suitable_crops, max_crops_per_land)
        else:
            selected_crops = suitable_crops  # 如果作物少于最大选择数量，则全部选择
        land_crop_mapping[land] = selected_crops
    return land_crop_mapping

# 第一季的作物与地块映射
land_crop_mapping_first_season = create_land_crop_mapping(land_names, crops_first_season)

# 第二季的作物与地块映射
land_crop_mapping_second_season = create_land_crop_mapping(land_names_second_season, crops_second_season)

In [5]:
# 优化函数，计算每个地块的种植方案
def optimize_land_crop(land_crop_mapping, crops, season, year):
    # 初始化决策变量和目标函数：优化目标是最大化净收入，并最小化因超产导致的滞销损失。
    decision_vars = {}
    objective_coeffs = []

    for land, selected_crops in land_crop_mapping.items():
        for crop in selected_crops:
            decision_vars[(crop, land)] = 0

            land_type = land_info_data[land_info_data['地块名称'] == land]['地块类型'].values[0]
            yield_data = planting_data[(planting_data['作物名称_x'] == crop) & (planting_data['地块类型'] == land_type)]['亩产量/斤'].values
            price_data = crop_sales_data[crop_sales_data['作物名称'] == crop]['销售单价/(元/斤)'].values
            cost_data = planting_data[(planting_data['作物名称_x'] == crop) & (planting_data['地块类型'] == land_type)]['种植成本/(元/亩)'].values

            if len(yield_data) > 0 and len(price_data) > 0 and len(cost_data) > 0:
                yield_per_acre = yield_data[0]
                price_per_unit = price_data[0]
                cost_per_acre = cost_data[0]
                
                # 获取2023年作物销售量
                sales_2023_volume = crop_sales_2023.get(crop, 0)
                
                # 计算实际销售收入
                net_revenue = (yield_per_acre * price_per_unit) - cost_per_acre
                
                # 计算滞销损失
                excess_volume = max(0, yield_per_acre - sales_2023_volume)
                wastage_loss = excess_volume * price_per_unit
                
                # 将净收入和滞销损失合并到目标函数中
                objective_coeffs.append(net_revenue - wastage_loss)
            else:
                objective_coeffs.append(0)

    objective_coeffs = np.array(objective_coeffs) * -1

    # 重新构建约束条件
    A_ub = []
    b_ub = []

    # 约束条件1：总面积约束
    for land in land_crop_mapping.keys():
        constraint = np.zeros(len(decision_vars))
        for i, (crop, land_name) in enumerate(decision_vars.keys()):
            if land_name == land:
                constraint[i] = 1
        A_ub.append(constraint)
        b_ub.append(land_info_data[land_info_data['地块名称'] == land]['地块面积/亩'].values[0])

    # 约束条件2：最小种植面积约束
    for land in land_crop_mapping.keys():
        min_area = 0.1 * land_info_data[land_info_data['地块名称'] == land]['地块面积/亩'].values[0]
        for crop in land_crop_mapping[land]:
            constraint = np.zeros(len(decision_vars))
            for i, (crop_name, land_name) in enumerate(decision_vars.keys()):
                if crop_name == crop and land_name == land:
                    constraint[i] = -1
            A_ub.append(constraint)
            b_ub.append(-min_area)

    # 约束条件3：三年内至少种植一次豆类作物
    bean_crops = crop_land_data[crop_land_data['作物类型'].str.contains('粮食（豆类）')]['作物名称'].tolist()
    for land in land_crop_mapping.keys():
        constraint = np.zeros(len(decision_vars))
        for i, (crop, land_name) in enumerate(decision_vars.keys()):
            if land_name == land and crop in bean_crops:
                constraint[i] = -1
        A_ub.append(constraint)
        b_ub.append(0)  # 三年内种植豆类作物的约束（具体实施时可能需要调整）

    # 约束条件4：销售量约束
    apply_sales_constraint = False
    if apply_sales_constraint:
        for crop in crops:
            total_yield = sum([
                decision_vars[(crop, land)] * planting_data[
                    (planting_data['作物名称_x'] == crop) & 
                    (planting_data['地块类型'] == land_info_data[land_info_data['地块名称'] == land]['地块类型'].values[0])
                ]['亩产量/斤'].values[0]
                for land in land_crop_mapping.keys() if (crop, land) in decision_vars
            ])
            constraint = np.zeros(len(decision_vars))
            for i, (crop_name, land_name) in enumerate(decision_vars.keys()):
                if crop_name == crop:
                    constraint[i] = 1
            A_ub.append(constraint)
            b_ub.append(crop_sales_2023[crop])  # 销售量约束

    A_ub = np.array(A_ub)
    b_ub = np.array(b_ub)

    # 优化模型
    result = linprog(c=objective_coeffs, A_ub=A_ub, b_ub=b_ub, method='highs')

    # 构建优化结果的表格输出
    if result.success:
        optimal_areas = result.x
        solution = {}
        
        for i, (crop, land) in enumerate(decision_vars.keys()):
            if land not in solution:
                solution[land] = {}
            solution[land][crop] = optimal_areas[i]

        # 确保结果表格中包含所有作物
        all_crops = sorted(set(crop_sales_data['作物名称'].tolist()))  # 所有作物的集合（去重并排序）
        
        # 构建结果表格
        results = pd.DataFrame(columns=['年', '季别', '地块名'] + all_crops)
        
        for land, crop_areas in solution.items():
            season_data = {'年': year, '季别': season, '地块名': land}
            for crop in all_crops:
                season_data[crop] = crop_areas.get(crop, 0)  # 如果该作物不在该地块中，则面积为0
            
            results = results._append(season_data, ignore_index=True)

        return results
    else:
        print(f"{year}year{season}faile.")
        return None

In [6]:
# 迭代计算 2024~2030 年的结果
years = list(range(2024, 2031))
all_results = []

for year in years:
    # 第一季
    results_first_season = optimize_land_crop(land_crop_mapping_first_season, crops_first_season, '第一季', year)
    # 第二季
    results_second_season = optimize_land_crop(land_crop_mapping_second_season, crops_second_season, '第二季', year)

    # 合并结果
    if results_first_season is not None and results_second_season is not None:
        combined_results = pd.concat([results_first_season, results_second_season], ignore_index=True)
        all_results.append(combined_results)

# 将所有结果写入Excel文件
if all_results:
    final_results = pd.concat(all_results, ignore_index=True)
    final_results.to_excel('2024至2030年农作物种植方案.xlsx', index=None)
                      

In [7]:
final_results

,年,季别,地块名,刀豆,包菜,南瓜,土豆,大白菜,大麦,小青菜,...,豇豆,辣椒,青椒,香菇,高粱,黄心菜,黄瓜,黄豆,黍子,黑豆
0,2024,第一季,A1,0,0,0,8.0,0,8.0,0,...,0,8.0,8.0,0,0,0,8.00,0,8.0,0
1,2024,第一季,A2,0,0,11.0,0.0,0,0.0,0,...,0,5.5,0.0,0,0,5.5,0.00,5.5,5.5,0
2,2024,第一季,A3,3.5,0,7.0,3.5,0,0.0,0,...,0,0.0,3.5,0,0,0,0.00,0,0.0,0
3,2024,第一季,A4,0,0,0,0.0,0,7.2,0,...,0,0.0,0.0,0,7.2,0,0.00,7.2,0.0,0
4,2024,第一季,A5,0,0,0,0.0,0,0.0,0,...,6.8,0.0,0.0,0,0,6.8,0.00,6.8,0.0,13.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
569,2030,第二季,E16,0,0.06,0,0,0.06,0,0,...,0.0,0.0,0,0.06,0,0,0.00,0,0,0
570,2030,第二季,F1,0.06,0,0,0,0.0,0,0,...,0.0,0.0,0,0,0,0,0.12,0,0,0
571,2030,第二季,F2,0,0,0,0.06,0.06,0,0.06,...,0.0,0.0,0.06,0,0,0,0.00,0,0,0
572,2030,第二季,F3,0.06,0,0,0,0.0,0,0.06,...,0.0,0.0,0,0.06,0,0,0.12,0,0,0


In [8]:
# 定义新的列名
new_columns = [
    '季别', '地块名', '黄豆', '黑豆', '红豆', '绿豆', '爬豆', '小麦', '玉米', '谷子', '高粱', '黍子', '荞麦', '南瓜',
    '红薯', '莜麦', '大麦', '水稻', '豇豆', '刀豆', '芸豆', '土豆', '西红柿', '茄子', '菠菜', '青椒', '菜花', '包菜',
    '油麦菜', '小青菜', '黄瓜', '生菜', '辣椒', '空心菜', '黄心菜', '芹菜', '大白菜', '白萝卜', '红萝卜', '榆黄菇',
    '香菇', '白灵菇', '羊肚菌'
]

with pd.ExcelWriter('./2024至2030年农作物种植方案Q1_1.xlsx', engine='xlsxwriter') as writer:
    for year in range(2024, 2031):
        year_data = final_results[final_results['年'] == year]
        
        year_data = year_data.drop(columns=['年'])
        
        year_data = year_data.reindex(columns=new_columns, fill_value=0)
    
        year_data.to_excel(writer, sheet_name=str(year), index=False)


## scipy.optimize.linprog 与线性规划（Linear Programming, LP）

`scipy.optimize.linprog` 使用的是**线性规划**（Linear Programming, LP）模型来进行数学求解。线性规划是求解线性目标函数在一组线性不等式或等式约束下的最优解的一类数学优化问题。

### 数学模型：线性规划（Linear Programming, LP）

线性规划的标准形式可以表示为：

### Minimize:
$$
c^T x
$$

**目标函数**是一个线性函数，其中：

- $c \in \mathbb{R}^n$ 是系数向量。
- $x \in \mathbb{R}^n$ 是决策变量向量。

### 约束条件分为以下几种：

1. **不等式约束**：

2. **等式约束**：

3. **变量范围约束**：

### 线性规划的求解原理

`scipy.optimize.linprog` 采用的求解方法是**单纯形法**（Simplex Method）或**内点法**（Interior Point Method）等优化算法。这些算法通过以下步骤来求解线性规划问题：

1. **初始化**
2. **迭代更新**
3. **检查收敛性**
4. **输出最优解**
